In [1]:
# Add parent directory and example_scripts to path to import strategy modules
import sys
import os
import io
from contextlib import redirect_stdout
import pandas as pd
from datetime import datetime
from pathlib import Path
import vectorbt as vbt

import warnings
warnings.filterwarnings('ignore')

# Get the absolute path to the parent directory and example_scripts
notebook_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
example_scripts_dir = os.path.join(parent_dir, 'example_scripts')

# Add both directories to Python path
sys.path.extend([parent_dir, example_scripts_dir])

# Now import the strategies
from strategy_framework import Strategy
from strategy_0_buy_and_hold import BuyAndHoldStrategy
from strategy_5_volatility_regime import VolatilityRegimeStrategy
from strategy_6_adaptive_trend import AdaptiveTrendStrategy


In [2]:
class BacktestConfig:
    def __init__(self, 
                 start_date=None, 
                 end_date=None,
                 rebalance_freq='1D',  # '1D' for daily, 'M' for monthly
                 initial_capital=100,
                 size=1.0,
                 size_type='percent'):
        self.start_date = pd.to_datetime(start_date) if start_date else None
        self.end_date = pd.to_datetime(end_date) if end_date else None
        self.rebalance_freq = rebalance_freq  # Can use 'M' directly for resampling
        self.initial_capital = initial_capital
        self.size = size
        self.size_type = size_type

    @classmethod
    def DAILY(cls):
        return cls(rebalance_freq='1D')
    
    @classmethod
    def MONTHLY(cls):
        return cls(rebalance_freq='M')

def load_data(config: BacktestConfig) -> pd.DataFrame:
    data_path = Path('..') / 'raw_data' / 'df.csv'
    df = pd.read_csv(data_path)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    
    # Filter by date range if specified
    if config.start_date:
        df = df[df.index >= config.start_date]
    if config.end_date:
        df = df[df.index <= config.end_date]
        
    return df

def create_portfolio(strategy, price, signals, config: BacktestConfig):
    # Convert signals to boolean if they're not already
    signals = signals.astype(bool)
    
    # Resample signals based on rebalance frequency
    if config.rebalance_freq != '1D':
        # Use 'M' for monthly resampling, then forward fill back to daily
        monthly_signals = signals.resample('M').last()
        signals = monthly_signals.reindex(price.index, method='ffill')
        # Ensure signals are boolean after resampling
        signals = signals.astype(bool)
    
    # Generate entries and exits
    entries = signals & ~signals.shift(1).fillna(False)
    exits = ~signals & signals.shift(1).fillna(False)
    
    # Create portfolio - always use '1D' for freq since we're using daily data
    return vbt.Portfolio.from_signals(
        price,
        entries,
        exits,
        freq='1D',  # Keep this as daily since we're using daily data
        init_cash=config.initial_capital,
        size=config.size,
        size_type=config.size_type,
        accumulate=False
    )

def format_results(stats_dict):
    # Convert to DataFrame and sort by Total Return
    df_stats = pd.DataFrame.from_dict(stats_dict, orient='index').T
    df_stats = df_stats.sort_values(by='Total Return [%]', axis=1, ascending=False)
    
    # Reorder rows to put new metrics after Total Return [%]
    ordered_rows = df_stats.index.tolist()
    total_return_idx = ordered_rows.index('Total Return [%]')
    ordered_rows.remove('Annualized Return [%]')
    ordered_rows.remove('Annualized Volatility [%]')
    ordered_rows.insert(total_return_idx + 1, 'Annualized Return [%]')
    ordered_rows.insert(total_return_idx + 2, 'Annualized Volatility [%]')
    df_stats = df_stats.reindex(ordered_rows)
    
    # Format specific columns
    formatted_df = df_stats.copy()
    
    # Format dates
    formatted_df.loc['Start'] = formatted_df.loc['Start'].apply(lambda x: pd.to_datetime(x).strftime('%m/%d/%Y'))
    formatted_df.loc['End'] = formatted_df.loc['End'].apply(lambda x: pd.to_datetime(x).strftime('%m/%d/%Y'))
    
    # Format percentage returns
    percentage_rows = ['Total Return [%]', 'Annualized Return [%]', 'Annualized Volatility [%]']
    for row in percentage_rows:
        formatted_df.loc[row] = formatted_df.loc[row].apply(
            lambda x: f"{x:.2f}%" if pd.notnull(x) else x
        )
    
    # Format Start Value and End Value to 2 decimal points
    for row in ['Start Value', 'End Value']:
        formatted_df.loc[row] = formatted_df.loc[row].apply(lambda x: f"{x:.2f}" if pd.notnull(x) else x)
    
    # Format all duration-related fields to just days
    duration_rows = ['Avg Winning Trade Duration', 'Avg Losing Trade Duration', 'Max Drawdown Duration', 'Period']
    for row in duration_rows:
        if row in formatted_df.index:
            formatted_df.loc[row] = formatted_df.loc[row].apply(
                lambda x: f"{pd.Timedelta(x).days} days" if pd.notnull(x) else x
            )
    
    # Round all other numeric values to 2 decimal points
    numeric_rows = [idx for idx in formatted_df.index 
                   if idx not in ['Start', 'End'] + percentage_rows + duration_rows + ['Start Value', 'End Value']]
    for row in numeric_rows:
        formatted_df.loc[row] = formatted_df.loc[row].apply(
            lambda x: f"{float(x):.2f}" if pd.notnull(x) and not isinstance(x, pd.Timedelta) else x
        )
    
    # Apply styling
    styled_df = formatted_df.style.set_properties(**{
        'text-align': 'center'
    }).set_table_styles([
        {'selector': 'th', 'props': [('text-align', 'center')]}
    ])
    
    return styled_df

def run_backtest(strategies, config: BacktestConfig):
    # Dictionary to store stats for each strategy
    all_stats = {}
    
    # Load and filter data once according to config
    df = load_data(config)
    
    # Create new instances of strategies with filtered data
    strategy_instances = [
        strategy.__class__(df) for strategy in strategies
    ]
    
    for strategy in strategy_instances:
        with redirect_stdout(io.StringIO()):
            signals = strategy.generate_signals()
        
        price = strategy.df[strategy.target_col]
        signals = signals.reindex(price.index)
        
        # Ensure signals are boolean
        if not isinstance(signals.dtype, pd.BooleanDtype):
            signals = signals.astype(bool)
        
        # Create portfolio using config
        pf = create_portfolio(strategy, price, signals, config)
        
        # Get portfolio stats
        stats_series = pf.stats()
        
        # Get returns stats
        returns = pf.returns()
        returns_stats = returns.vbt.returns(freq='1D', year_freq='365D')
        
        # Add annualized metrics
        stats_series['Annualized Return [%]'] = returns_stats.annualized() * 100
        stats_series['Annualized Volatility [%]'] = returns_stats.annualized_volatility() * 100
        
        all_stats[strategy.__class__.__name__] = stats_series
    
    return format_results(all_stats)

# Initialize strategies with full data first
df = load_data(BacktestConfig())  # Load full dataset
strategies = [
    BuyAndHoldStrategy(df),
    VolatilityRegimeStrategy(df),
    AdaptiveTrendStrategy(df),
]

# Create configs for different periods
config_default = BacktestConfig()
results_default = run_backtest(strategies, config_default)

# Test 2020 period with monthly rebalancing
config_2020 = BacktestConfig(
    start_date='2009-01-01',
    end_date='2025-12-31',
    rebalance_freq='M'
)
results_2020 = run_backtest(strategies, config_2020)

# Display results
display(results_default)
display(results_2020)

,AdaptiveTrendStrategy,BuyAndHoldStrategy,VolatilityRegimeStrategy
Start,10/31/2002,10/31/2002,10/31/2002
End,12/27/2024,12/27/2024,12/27/2024
Period,5562 days,5562 days,5562 days
Start Value,100.00,100.00,100.00
End Value,155.22,134.69,109.89
Total Return [%],55.22%,34.69%,9.89%
Annualized Return [%],2.93%,1.97%,0.62%
Annualized Volatility [%],1.21%,1.82%,0.41%
Benchmark Return [%],34.69,34.69,34.69
Max Gross Exposure [%],100.00,100.00,100.00


,AdaptiveTrendStrategy,BuyAndHoldStrategy,VolatilityRegimeStrategy
Start,01/02/2009,01/02/2009,01/02/2009
End,12/27/2024,12/27/2024,12/27/2024
Period,4019 days,4019 days,4019 days
Start Value,100.00,100.00,100.00
End Value,165.02,150.27,103.90
Total Return [%],65.02%,50.27%,3.90%
Annualized Return [%],4.65%,3.77%,0.35%
Annualized Volatility [%],1.19%,1.87%,0.50%
Benchmark Return [%],50.27,50.27,50.27
Max Gross Exposure [%],100.00,100.00,100.00
